# Project Summary
### Overview : 
This project aims to build up classification models for the cat and dog owners from text comments. There are around 6,000,000 comments recorded in the dataset and the dataset itself is unlabeled. Each data row will have an userid, an text comment and belongs to one channel.

### Dataset : 
The dataset is acquired from GitHub. https://drive.google.com/file/d/1dZk9s3qi3WwZMlgcH8GA3sP2QfJWyB8c/view?usp=sharing

There are around 6,000,000 comments recorded in the dataset and the dataset itself is unlabeled. The dataset is comma separated, with a header line defining the attribute names, listed here:

  creator_name: Name of the Youtube channel creator;
  userid: Integer identifier for the users commenting on the Youtube channels;
  comment: Text of the comments made by the users.

### Details :
1. Searched specific terms which a cat/dog owner might have, and label those users as dog&cat owners. Also labeled users who don't have pets as those whose comments don't contain any specific terms. By this way, we turn our dataset into labeled ones and convert problem from unsupervised to supervised.
2. Use RegexTokenizer to tokenize text comments.

 RegexTokenizer allows more advanced tokenization based on regular expression (regex) matching. By default, the parameter “pattern” is used as delimiters to split the input text. Alternatively, users can set parameter “gaps” to false indicating the regex “pattern” denotes “tokens” rather than splitting gaps, and find all matching occurrences as the tokenization result.)

3. Use Word2Vec represent text features as vectors.
4. Train Logistic regression model and random forest model to classify audiences. Evaluation metrics are include: precision, recall, accuracy, AUC. Based on my analysis (I only randomly pick a small portion of dataset for training in order to speed process up) random forest outperforms LR model.
5. Use our trained model to predict all users in the dataset. And get statistical sense of how dog&cat owners' distribution. Extract word frequency to see related topics regards to those owner.

## 0. Data Exploration and Cleaning

In [3]:
df = spark.read.load("/FileStore/tables/animals_comments.csv", format='csv', header = True, inferSchema = True)
df.show(10)

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows

In [4]:
df.dtypes

Out[6]: [('creator_name', 'string'), ('userid', 'double'), ('comment', 'string')]

In [5]:
print("Number of rows in df:", df.count())

Number of rows in df: 5820035

In [6]:
# Count null values in each columns 
print('Number of null values in creator_name: ',df.filter(df['creator_name'].isNull()).count())
print('Number of null values in userid: ',df.filter(df['userid'].isNull()).count())
print('Number of null values in comment: ',df.filter(df['comment'].isNull()).count())

Number of null values in creator_name: 32050
Number of null values in userid: 565
Number of null values in comment: 1051

In [7]:
#drop out rows with no comments and no userid
def pre_process(df):
  df_drop = df.filter(df['comment'].isNotNull())
  df_drop = df_drop.filter(df_drop['userid'].isNotNull())
  df_drop = df_drop.dropDuplicates()
  
  print('After dropping, we have ', str(df_drop.count()), 'row in dataframe')
  return df_drop

df_drop = pre_process(df)
5820035

After dropping, we have 5757212 row in dataframe
Out[50]: 5820035

In [8]:
import pyspark.sql.functions as F
#convert text in comment to lower case.
df_clean = df_drop.withColumn('comment', F.lower(F.col('comment')))

In [9]:
display(df_clean)

creator_name,userid,comment
Brian Barczyk,28322.0,since i watched these snake videos i am not afraid of snakes anymore and now i really want a snake for my birthday
WaysideWaifs,89939.0,why are humans so inhuman...
TobyTurner,236731.0,roses are red this makes no sense i got a science that i havemicro wave
meow meow,260511.0,..omai-wa.*rejectooooooo!!!*
Brave Wilderness,420830.0,good job on getting on trending
꼬부기아빠 My Pet Diary,459543.0,2!!0!!!만명!!!!! 축하드려용!!!
The Dodo,485503.0,who gave thumbs down to this video? must be a person without heart
Hope For Paws - Official Rescue Channel,504270.0,such an intense rescue! i felt i could feel your emotions and it was draining!!god bless lisa on her trip to her new home!!!
Viktor Larkhill,542233.0,oh that poor cat!!
Brave Wilderness,543795.0,you can live your live free by traveling the unknown! lets travel then!


#### This is an unlabeled dataset and we want to train a clasifier to identify cat and dog owners. Thus first thing to do is to label each comment.
1. Label comment when he/she has dogs or cats.
2. label comment when he/she don't have a dog or cat.
3. Combine 1 and 2 as our training dataset, and rest of the dataset will be the data we predict.
4. The strategy to tell if a user own or not own is using key words (such as 'I have a dog') to tell. We don't have enough time to check the user information one by one.

In [11]:
# find user with preference of dog and cat
cond = (df_clean["comment"].like("%my dog%") | df_clean["comment"].like("%i have a dog%")\
        | df_clean["comment"].like("%my cat%") | df_clean["comment"].like("%i have a cat%") \
        | df_clean["comment"].like("%my dogs%") | df_clean["comment"].like("%my cats%")\
        | df_clean["comment"].like("%my cat%") | df_clean["comment"].like("%i have dogs%")\
        | df_clean["comment"].like("%i have cats%") | df_clean["comment"].like("%my puppy%")\
        | df_clean["comment"].like("%my kitten%") | df_clean["comment"].like("%i have a puppy%")\
        | df_clean["comment"].like("%i have puppies%"))

df_clean = df_clean.withColumn('dog_cat',  cond)

# find user do not have 
df_clean = df_clean.withColumn('no_pet', ~df_clean["comment"].like("%my%") & ~df_clean["comment"].like("%have%") & ~df_clean["comment"].like("%my dog%") \
                              & ~df_clean["comment"].like("%my cat%")) 
display(df_clean)

creator_name,userid,comment,dog_cat,no_pet
Brian Barczyk,28322.0,since i watched these snake videos i am not afraid of snakes anymore and now i really want a snake for my birthday,false,false
WaysideWaifs,89939.0,why are humans so inhuman...,false,true
TobyTurner,236731.0,roses are red this makes no sense i got a science that i havemicro wave,false,false
meow meow,260511.0,..omai-wa.*rejectooooooo!!!*,false,true
Brave Wilderness,420830.0,good job on getting on trending,false,true
꼬부기아빠 My Pet Diary,459543.0,2!!0!!!만명!!!!! 축하드려용!!!,false,true
The Dodo,485503.0,who gave thumbs down to this video? must be a person without heart,false,true
Hope For Paws - Official Rescue Channel,504270.0,such an intense rescue! i felt i could feel your emotions and it was draining!!god bless lisa on her trip to her new home!!!,false,true
Viktor Larkhill,542233.0,oh that poor cat!!,false,true
Brave Wilderness,543795.0,you can live your live free by traveling the unknown! lets travel then!,false,true


### 1. Build the classifier

In [13]:
# data preprocessing 
from pyspark.ml.feature import RegexTokenizer

regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="text", pattern="\\W")
df_clean = regexTokenizer.transform(df_clean)
df_clean.show(10)

# '\W' Matches any character which is not a Unicode word character. This is the opposite of \w. If the ASCII flag is used this becomes the equivalent of [^a-zA-Z0-9_] (but the flag affects the entire regular expression, so in such cases using an explicit [^a-zA-Z0-9_] may be a better choice).

+-----------------------+--------+------------------------+-------+------+--------------------+
 creator_name| userid| comment|dog_cat|no_pet| text|
+-----------------------+--------+------------------------+-------+------+--------------------+
 Brian Barczyk| 28322.0| since i watched t...| false| false|[since, i, watche...|
 WaysideWaifs| 89939.0| why are humans so...| false| true|[why, are, humans...|
 TobyTurner|236731.0| roses are red thi...| false| false|[roses, are, red,...|
 meow meow|260511.0| ..omai-wa.*reject...| false| true|[omai, wa, reject...|
 Brave Wilderness|420830.0| good job on getti...| false| true|[good, job, on, g...|
꼬부기아빠 My Pet Diary|459543.0|2!!0!!!만명!!!!! 축하...| false| true| [2, 0]|
 The Dodo|485503.0| who gave thumbs d...| false| true|[who, gave, thumb...|
 Hope For Paws - O...|504270.0| such an intense r...| false| true|[such, an, intens...|
 Viktor Larkhill|542233.0| oh that poor cat!!| false| true|[oh, that, poor, ...|
 Brave Wilderness|543795.0| you can live your...| false| true|[you, can, live, ...|
+-----------------------+--------+------------------------+-------+------+--------------------+
only showing top 10 rows

In [14]:
#Only select 1,000,000 rows for testing
from pyspark.sql.functions import rand 

df_clean.orderBy(rand(seed=0)).createOrReplaceTempView("table1")
df_clean = spark.sql("select * from table1 limit 1000000")

In [15]:
from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Display some of the stop words
stopwords[:10]

Out[56]: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [16]:
remover.setInputCol("text")
remover.setOutputCol("vector_no_stopw")

df_clean = remover.transform(df_clean)
df_clean.show(10)

+--------------------+---------+--------------------+-------+------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pet| text| vector_no_stopw|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+
 nywolforg| 472090.0|omg they sleep ju...| true| false|[omg, they, sleep...|[omg, sleep, like...|
Hope For Paws - O...|2114449.0|de verdad que soi...| false| true|[de, verdad, que,...|[de, verdad, que,...|
 Info Marvel| 185789.0|no creo que metan...| false| true|[no, creo, que, m...|[creo, que, metan...|
 Viktor Larkhill|1589914.0| alive| false| true| [alive]| [alive]|
 Caelyn Le|2339310.0|love it. cant wai...| false| true|[love, it, cant, ...|[love, cant, wait...|
 RaleighLink14|1582365.0|i feel like if yo...| false| true|[i, feel, like, i...|[feel, like, want...|
 peta2TV| 346689.0| how can i help?!| false| true| [how, can, i, help]| [help]|
 Cole & Marmalade| 723089.0| meowzies!!!| false| true| [meowzies]| [meowzies]|
 GOOD BYE|2033233.0|beyoncé is so ove...| false| true|[beyonc, is, so, ...| [beyonc, overrated]|
 Brave Wilderness|1350178.0|the music makes i...| false| true|[the, music, make...|[music, makes, in...|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+
only showing top 10 rows

In [17]:
# use word2vec get text vector feature.
from pyspark.ml.feature import Word2Vec
# Learn a mapping from words to Vectors. (choose higher vectorSize here)
word2Vec = Word2Vec(vectorSize=20, minCount=1, inputCol="vector_no_stopw", outputCol="wordVector")
#word2Vec = Word2Vec(vectorSize=50, minCount=1, inputCol="text", outputCol="wordVector")
model = word2Vec.fit(df_clean)

df_model = model.transform(df_clean)
df_model.show(10)

+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pet| text| vector_no_stopw| wordVector|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
 nywolforg| 472090.0|omg they sleep ju...| true| false|[omg, they, sleep...|[omg, sleep, like...|[0.03663111105561...|
Hope For Paws - O...|2114449.0|de verdad que soi...| false| true|[de, verdad, que,...|[de, verdad, que,...|[0.17581055359914...|
 Info Marvel| 185789.0|no creo que metan...| false| true|[no, creo, que, m...|[creo, que, metan...|[0.32407902525021...|
 Viktor Larkhill|1589914.0| alive| false| true| [alive]| [alive]|[0.24931317567825...|
 Caelyn Le|2339310.0|love it. cant wai...| false| true|[love, it, cant, ...|[love, cant, wait...|[0.09205778694319...|
 RaleighLink14|1582365.0|i feel like if yo...| false| true|[i, feel, like, i...|[feel, like, want...|[0.36573381721973...|
 peta2TV| 346689.0| how can i help?!| false| true| [how, can, i, help]| [help]|[0.09843914210796...|
 Cole & Marmalade| 723089.0| meowzies!!!| false| true| [meowzies]| [meowzies]|[0.01666151545941...|
 GOOD BYE|2033233.0|beyoncé is so ove...| false| true|[beyonc, is, so, ...| [beyonc, overrated]|[-0.0390380378812...|
 Brave Wilderness|1350178.0|the music makes i...| false| true|[the, music, make...|[music, makes, in...|[0.21634679287672...|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
only showing top 10 rows

#### Get train and test dataset.
We will only use the observations of which the user we are sure about (dog/cat owner or no-pet-owner) to form the train and test dataset.
The rest observations will be served as dataset for predicting.

In [19]:
df_pets = df_model.filter(F.col('dog_cat') == True) 
df_no_pets = df_model.filter(F.col('no_pet') ==  True)
print("Number of confirmed user who own dogs or cats: ", df_pets.count())
print("Number of confirmed user who don't have pet's: ", df_no_pets.count())

Number of confirmed user who own dogs or cats: 9229
Number of confirmed user who don't have pet's: 832763

In [20]:
df_pets.show() 

+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pet| text| vector_no_stopw| wordVector|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
 nywolforg| 472090.0|omg they sleep ju...| true| false|[omg, they, sleep...|[omg, sleep, like...|[0.03663111105561...|
 Gardea23|1346087.0|hey matt my puppy...| true| false|[hey, matt, my, p...|[hey, matt, puppy...|[0.05912114121019...|
 The Dodo| 48857.0|my dog died yeste...| true| false|[my, dog, died, y...|[dog, died, yeste...|[0.18074695020914...|
 Vet Ranch|1162567.0|my dog chewy is a...| true| false|[my, dog, chewy, ...|[dog, chewy, olde...|[0.13598554590889...|
 The Dodo| 297832.0|i want them all!t...| true| false|[i, want, them, a...|[want, therere, c...|[0.08566602226346...|
 DarkDynastyK9s| 91432.0|hi im really inte...| true| false|[hi, im, really, ...|[hi, im, really, ...|[0.04976289109166...|
 Happy Tails|1790011.0|my cats name is r...| true| false|[my, cats, name, ...| [cats, name, reimi]|[-0.1439464638630...|
Stray Rescue of S...| 471458.0|he looks exactly ...| true| false|[he, looks, exact...|[looks, exactly, ...|[-0.0613207606864...|
 TobyTurner| 953315.0|my cats name was ...| true| false|[my, cats, name, ...| [cats, name, kitty]|[-0.1770278662443...|
 Think Like A Horse| 581653.0|my cat has learne...| true| false|[my, cat, has, le...|[cat, learned, cl...|[0.10445102937519...|
Worlds Fuzziest V...| 550829.0|0:02 she seems so...| true| false|[0, 02, she, seem...|[0, 02, seems, me...|[0.15361591518201...|
 The Dodo| 505381.0|oooh that hilario...| true| false|[oooh, that, hila...|[oooh, hilarious,...|[0.05270984955132...|
 Vet Ranch|2474101.0|when we rescued m...| true| false|[when, we, rescue...|[rescued, kitten,...|[0.04845917345955...|
 stacyvlogs|2208271.0|i never vaccinate...| true| false|[i, never, vaccin...|[never, vaccinate...|[0.13945544500731...|
 Dogumentary TV| 242960.0|i had a pet germa...| true| false|[i, had, a, pet, ...|[pet, german, she...|[0.14301053462550...|
 Rachel Fusaro| 927355.0|i would bring my ...| true| false|[i, would, bring,...|[bring, dog, home...|[0.01957209408283...|
 Taylor Nicole Dean|2047724.0|milo is my cats n...| true| false|[milo, is, my, ca...|[milo, cats, name...|[-0.0545608783140...|
 Stonnie Dennis|2195639.0|i just love your ...| true| false|[i, just, love, y...|[love, videos, lo...|[0.06932048271927...|
 stacyvlogs|1664227.0|dang im really la...| true| false|[dang, im, really...|[dang, im, really...|[0.11546921926109...|
 stacyvlogs| 20526.0|we got my dog fro...| true| false|[we, got, my, dog...|[got, dog, amish,...|[0.08467005255132...|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [21]:
df_no_pets.show(10)

+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
 creator_name| userid| comment|dog_cat|no_pet| text| vector_no_stopw| wordVector|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
Hope For Paws - O...|2114449.0|de verdad que soi...| false| true|[de, verdad, que,...|[de, verdad, que,...|[0.17581055359914...|
 Info Marvel| 185789.0|no creo que metan...| false| true|[no, creo, que, m...|[creo, que, metan...|[0.32407902525021...|
 Viktor Larkhill|1589914.0| alive| false| true| [alive]| [alive]|[0.24931317567825...|
 Caelyn Le|2339310.0|love it. cant wai...| false| true|[love, it, cant, ...|[love, cant, wait...|[0.09205778694319...|
 RaleighLink14|1582365.0|i feel like if yo...| false| true|[i, feel, like, i...|[feel, like, want...|[0.36573381721973...|
 peta2TV| 346689.0| how can i help?!| false| true| [how, can, i, help]| [help]|[0.09843914210796...|
 Cole & Marmalade| 723089.0| meowzies!!!| false| true| [meowzies]| [meowzies]|[0.01666151545941...|
 GOOD BYE|2033233.0|beyoncé is so ove...| false| true|[beyonc, is, so, ...| [beyonc, overrated]|[-0.0390380378812...|
 Brave Wilderness|1350178.0|the music makes i...| false| true|[the, music, make...|[music, makes, in...|[0.21634679287672...|
 Brave Wilderness| 321227.0|guys pain toleran...| false| true|[guys, pain, tole...|[guys, pain, tole...|[0.37581439875066...|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+
only showing top 10 rows

Note that number of negative labels is around 100 times more than positive labels, so here we need to downsampling the negative labels. By rule of thumb, the gap should be no more than 10 times. But here I make them balance to the ratio aroudn 1:2 (1 for positive: 2 for negative)

In [23]:
from pyspark.sql.functions import rand 
df_no_pets.orderBy(rand()).createOrReplaceTempView("table")

Num_Pos_Label = df_model.filter(F.col('dog_cat') == True).count() 
Num_Neg_Label = df_model.filter(F.col('no_pet') ==  True).count()

#Q1 = spark.sql("SELECT col1 from table where col2>500 limit {}, 1".format(q25))
#pass variable to sql
df_no_pets_down = spark.sql("select * from table where limit {}".format(Num_Pos_Label*2))


In [24]:
positive:nagative = 1:2
print('Now after balancing the lables, we have ')   
print('Positive label: ', Num_Pos_Label)
print('Negtive label: ', df_no_pets_down.count())

Now after balancing the lables, we have 
Positive label: 9229
Negtive label: 18458

In [25]:
#combine the datasets
def get_label(df_pets,df_no_pets_down):
  df_labeled = df_pets.select('dog_cat','wordVector').union(df_no_pets_down.select('dog_cat','wordVector'))
  return df_labeled

df_labeled = get_label(df_pets,df_no_pets_down)
df_labeled.show(10)

+-------+--------------------+
dog_cat| wordVector|
+-------+--------------------+
 true|[0.03663111105561...|
 true|[0.05912114121019...|
 true|[0.18074695020914...|
 true|[0.13598554590889...|
 true|[0.08566602226346...|
 true|[0.04976289109166...|
 true|[-0.1439464638630...|
 true|[-0.0613207606864...|
 true|[-0.1770278662443...|
 true|[0.10445102937519...|
+-------+--------------------+
only showing top 10 rows

In [26]:
#convert Boolean value to 1 and 0's
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

def multiple(x):
  return int(x*1)
udf_boolToInt= udf(lambda z: multiple(z),IntegerType())
df_labeled = df_labeled.withColumn('label',udf_boolToInt('dog_cat'))
df_labeled.show(10)

+-------+--------------------+-----+
dog_cat| wordVector|label|
+-------+--------------------+-----+
 true|[0.03663111105561...| 1|
 true|[0.05912114121019...| 1|
 true|[0.18074695020914...| 1|
 true|[0.13598554590889...| 1|
 true|[0.08566602226346...| 1|
 true|[0.04976289109166...| 1|
 true|[-0.1439464638630...| 1|
 true|[-0.0613207606864...| 1|
 true|[-0.1770278662443...| 1|
 true|[0.10445102937519...| 1|
+-------+--------------------+-----+
only showing top 10 rows

In [27]:
#see whether agg reasonable (optional)
# from pyspark.sql.functions import col, countDistinct
# df_labeled.agg(countDistinct(col("label")).alias("count")).show()
# df_labeled.select('dog_cat').distinct().count()

### a. Logistic Regression Model

In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

train, test = df_labeled.randomSplit([0.8, 0.2], seed=12345)

lr = LogisticRegression(featuresCol="wordVector",labelCol="label" , maxIter=10, regParam=0.1, elasticNetParam=0.8)

# Run TrainValidationSplit, and choose the best set of parameters.
lrModel = lr.fit(train)

# Make predictions on test data. model is the model with combination of parameters
# that performed best.
predictions = lrModel.transform(test)
predictions.show(10)

+-------+--------------------+-----+--------------------+--------------------+----------+
dog_cat| wordVector|label| rawPrediction| probability|prediction|
+-------+--------------------+-----+--------------------+--------------------+----------+
 true|[-0.4318252503871...| 1|[1.27929885253951...|[0.78233040197739...| 0.0|
 true|[-0.4080661647021...| 1|[-0.0479790411062...|[0.48800754017710...| 1.0|
 true|[-0.3836099505424...| 1|[1.21066745093337...|[0.77041702549228...| 0.0|
 true|[-0.3356285219391...| 1|[1.04724446747693...|[0.74024541034282...| 0.0|
 true|[-0.3328908085823...| 1|[1.40116181929665...|[0.80236818700978...| 0.0|
 true|[-0.3327309464414...| 1|[0.78274558110932...|[0.68627154764386...| 0.0|
 true|[-0.3025755994021...| 1|[0.77506274559819...|[0.68461504833950...| 0.0|
 true|[-0.2954121615205...| 1|[-0.7969181384991...|[0.31068514460307...| 1.0|
 true|[-0.2945575465758...| 1|[0.18190334868354...|[0.54535085552370...| 0.0|
 true|[-0.2856480909718...| 1|[-0.3275387819786...|[0.41883959564357...| 1.0|
+-------+--------------------+-----+--------------------+--------------------+----------+
only showing top 10 rows

In [30]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

# # Obtain the objective per iteration
# objectiveHistory = trainingSummary.objectiveHistory
# print("objectiveHistory:")
# for objective in objectiveHistory:
#     print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
# print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# # Set the model threshold to maximize F-Measure
# fMeasure = trainingSummary.fMeasureByThreshold
# maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
# bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
#     .select('threshold').head()['threshold']
# lr.setThreshold(bestThreshold)

+--------------------+--------------------+
 FPR| TPR|
+--------------------+--------------------+
 0.0| 0.0|
0.007190340523673857|0.020100502512562814|
0.010378510378510378| 0.04400380279777265|
0.014109347442680775| 0.06464756213499932|
 0.01824718491385158| 0.08461225044139617|
 0.02184235517568851| 0.10512019557245687|
0.025234025234025233| 0.12671465435284532|
 0.02862569529236196| 0.14749422789623795|
 0.03242436575769909| 0.16759473040880077|
 0.03629086962420296| 0.18851011815835936|
 0.03927553927553928| 0.20996876273258183|
0.043345543345543344| 0.2291185658019829|
0.045787545787545784| 0.25179953823169904|
 0.04931488264821598| 0.27230748336275973|
 0.05304571971238638| 0.2922721716691566|
0.056505223171889836| 0.31291593100638326|
 0.05982905982905983| 0.3336955045497759|
 0.06423823090489757| 0.3523020507945131|
 0.06762990096323429| 0.3729458101317398|
 0.07081807081807082| 0.39454026891212823|
+--------------------+--------------------+
only showing top 20 rows

In [31]:
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

areaUnderROC: 0.8673789201427391

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


def get_evaluation_result(predictions):
  evaluator = BinaryClassificationEvaluator(
      labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
  AUC = evaluator.evaluate(predictions)

  TP = predictions[(predictions["label"] == 1) & (predictions["prediction"] == 1.0)].count()
  FP = predictions[(predictions["label"] == 0) & (predictions["prediction"] == 1.0)].count()
  TN = predictions[(predictions["label"] == 0) & (predictions["prediction"] == 0.0)].count()
  FN = predictions[(predictions["label"] == 1) & (predictions["prediction"] == 0.0)].count()

  accuracy = (TP + TN)*1.0 / (TP + FP + TN + FN)
  precision = TP*1.0 / (TP + FP)
  recall = TP*1.0 / (TP + FN)


  print ("True Positives:", TP)
  print ("False Positives:", FP)
  print ("True Negatives:", TN)
  print ("False Negatives:", FN)
  print ("Test Accuracy:", accuracy)
  print ("Test Precision:", precision)
  print ("Test Recall:", recall)
  print ("Test AUC of ROC:", AUC)

print("Prediction result summary for Logistic Regression Model:  ")
get_evaluation_result(predictions)

Prediction result summary for Logistic Regression Model: 
True Positives: 426
False Positives: 183
True Negatives: 3533
False Negatives: 1440
Test Accuracy: 0.709243998566822
Test Precision: 0.6995073891625616
Test Recall: 0.2282958199356913
Test AUC of ROC: 0.8616747542852341

### b. Random Forest Model

In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="wordVector", numTrees=15)

# Train model.  This also runs the indexers.
model = rf.fit(train)

# Make predictions.
predictions = model.transform(test)

# Select example rows to display.
predictions.show(10)

+-------+--------------------+-----+--------------------+--------------------+----------+
dog_cat| wordVector|label| rawPrediction| probability|prediction|
+-------+--------------------+-----+--------------------+--------------------+----------+
 true|[-0.4318252503871...| 1|[13.1179211681203...|[0.87452807787469...| 0.0|
 true|[-0.4080661647021...| 1|[5.42228180519314...|[0.36148545367954...| 1.0|
 true|[-0.3836099505424...| 1|[13.0449201089145...|[0.86966134059430...| 0.0|
 true|[-0.3356285219391...| 1|[11.9554343457804...|[0.79702895638536...| 0.0|
 true|[-0.3328908085823...| 1|[13.4434566764345...|[0.89623044509563...| 0.0|
 true|[-0.3327309464414...| 1|[13.1094591798065...|[0.87396394532043...| 0.0|
 true|[-0.3025755994021...| 1|[9.72183711293967...|[0.64812247419597...| 0.0|
 true|[-0.2954121615205...| 1|[3.86348287690549...|[0.25756552512703...| 1.0|
 true|[-0.2945575465758...| 1|[5.73850901591776...|[0.38256726772785...| 1.0|
 true|[-0.2856480909718...| 1|[7.75232539218832...|[0.51682169281255...| 0.0|
+-------+--------------------+-----+--------------------+--------------------+----------+
only showing top 10 rows

In [35]:
print("Prediction result summary for Random Forest Model:  ")
get_evaluation_result(predictions)



Prediction result summary for Random Forest Model: 
True Positives: 1187
False Positives: 293
True Negatives: 3423
False Negatives: 679
Test Accuracy: 0.8258688642063776
Test Precision: 0.802027027027027
Test Recall: 0.6361200428724545
Test AUC of ROC: 0.9075520878400731

### Model Performance Evaluation
#####LR:
Test Accuracy: 0.709243998566822
Test Precision: 0.6995073891625616
Test Recall: 0.2282958199356913
Test AUC of ROC: 0.8616747542852341
#####RF:
Test Accuracy: 0.8258688642063776
Test Precision: 0.802027027027027
Test Recall: 0.6361200428724545
Test AUC of ROC: 0.9075520878400731

#####Conclusion: RF performs better than LR. (RF has higher precision and recall.)

### Classify All The Users
We can now apply the cat/dog classifiers to all the other users in the dataset.

In [38]:
# get dataset for prediction (note to exclude people we already know the label)
# Users we don't know yet are those who don't own dog&cat and no_pets attribute is also flase
df_unknow = df_model.filter((F.col('dog_cat') == False) & (F.col('no_pet') == False)) 
df_unknow = df_unknow.withColumn('label',df_unknow.dog_cat.cast('integer'))
print("There are {} users whose attribute is unclear.".format(df_unknow.count()))
pred_all = model.transform(df_unknow)
pred_all.show(10)

There are 158008 users whose attribute is unclear.
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 creator_name| userid| comment|dog_cat|no_pet| text| vector_no_stopw| wordVector|label| rawPrediction| probability|prediction|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
 Brave Wilderness|1405480.0|hey guys my name ...| false| false|[hey, guys, my, n...|[hey, guys, name,...|[0.27014118582010...| 0|[13.1952460766595...|[0.87968307177730...| 0.0|
 Talking Kitty Cat| 482275.0|that kitten looks...| false| false|[that, kitten, lo...|[kitten, looks, l...|[-0.1185142789036...| 0|[5.79794119128188...|[0.38652941275212...| 1.0|
 Brian Barczyk| 732406.0|bryan i love the ...| false| false|[bryan, i, love, ...|[bryan, love, ide...|[0.12536594153406...| 0|[12.4158419334418...|[0.82772279556278...| 0.0|
 Brian Barczyk| 63870.0|brian i love ya b...| false| false|[brian, i, love, ...|[brian, love, ya,...|[0.24020944908261...| 0|[11.8519233669001...|[0.79012822446001...| 0.0|
 Brave Wilderness| 700415.0|isnt he afraid to...| false| false|[isnt, he, afraid...|[isnt, afraid, st...|[0.23243867925537...| 0|[9.55338496939142...|[0.63689233129276...| 0.0|
 Brian Barczyk| 993751.0|nawww dont worry!...| false| false|[nawww, dont, wor...|[nawww, dont, wor...|[0.14454973785376...| 0|[11.9638981678726...|[0.79759321119150...| 0.0|
Hope For Paws - O...|1534859.0|what a sweet girl...| false| false|[what, a, sweet, ...|[sweet, girl, bre...|[0.10540573248782...| 0|[7.66416361540800...|[0.5109442410272,...| 0.0|
 dzlfreek| 653807.0|hey great videos....| false| false|[hey, great, vide...|[hey, great, vide...|[0.15356377613815...| 0|[11.6257830326126...|[0.77505220217417...| 0.0|
Deadly Tarantula ...|2041195.0|do you have any s...| false| false|[do, you, have, a...|[scolopendra, her...|[0.06518665479961...| 0|[13.0428571902090...|[0.86952381268060...| 0.0|
 Reptile Channel| 567073.0|i love these but ...| false| false|[i, love, these, ...|[love, makes, cry...|[0.23269294316952...| 0|[4.99607718096951...|[0.33307181206463...| 1.0|
+--------------------+---------+--------------------+-------+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
only showing top 10 rows

##### Fraction of the users who are cat/dog owners (ML estimate):

(Num of owner labeled + Num of owner predicted) / Total users in our used dataset

In [40]:
#df.select("columnname").distinct().show()

#number of total user
total_user = df_model.select('userid').distinct().count()
#number of labeled owner
owner_labeled = df_pets.select('userid').distinct().count() 
#number of owner predicted
owner_pred = pred_all.filter(F.col('prediction') == 1.0).count()

fraction = (owner_labeled+owner_pred)/total_user
print('Fraction of the users who are cat/dog owners (ML estimate): ', round(fraction,3))

Fraction of the users who are cat/dog owners (ML estimate): 0.056

### 3. Gain insights about what cat/dog owners care

In [42]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml.clustering import LDA
import pyspark.sql.functions as f

#Combine df_pets (Num of owner labeled) and predicts (Num of owner predicted)
df_all_owner = df_pets.select('vector_no_stopw').union(pred_all.filter(F.col('prediction') == 1.0).select('vector_no_stopw'))

#TF model
count_vec = CountVectorizer(inputCol = 'vector_no_stopw',outputCol = 'rawFeatures',vocabSize = 123241, minDF = 2.0)
count_vec_model = count_vec.fit(df_all_owner)
featurizedData = count_vec_model.transform(df_all_owner)
vocab = count_vec_model.vocabulary

#IDF model
idf = IDF(inputCol = 'rawFeatures',outputCol = 'features',minDocFreq=2)
idfmodel = idf.fit(featurizedData)
rescaledData = idfmodel.transform(featurizedData)
rescaledData.cache()


Out[79]: DataFrame[vector_no_stopw: array<string>, rawFeatures: vector, features: vector]

In [43]:
rescaledData.show(10)

+--------------------+--------------------+--------------------+
 vector_no_stopw| rawFeatures| features|
+--------------------+--------------------+--------------------+
[omg, sleep, like...|(18151,[1,4,254,2...|(18151,[1,4,254,2...|
[hey, matt, puppy...|(18151,[85,93,252...|(18151,[85,93,252...|
[dog, died, yeste...|(18151,[0,85,551]...|(18151,[0,85,551]...|
[dog, chewy, olde...|(18151,[0,9,19,21...|(18151,[0,9,19,21...|
[want, therere, c...|(18151,[3,4,16,62...|(18151,[3,4,16,62...|
[hi, im, really, ...|(18151,[0,3,5,8,9...|(18151,[0,3,5,8,9...|
 [cats, name, reimi]|(18151,[11,115],[...|(18151,[11,115],[...|
[looks, exactly, ...|(18151,[0,1,87,35...|(18151,[0,1,87,35...|
 [cats, name, kitty]|(18151,[11,115,24...|(18151,[11,115,24...|
[cat, learned, cl...|(18151,[4,5,13,91...|(18151,[4,5,13,91...|
+--------------------+--------------------+--------------------+
only showing top 10 rows

In [44]:
#LDA
lda = LDA(k=10, maxIter=10)
lda_model = lda.fit(rescaledData.select('features'))
topics = lda_model.describeTopics(10)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)
# Shows the result
transformed = lda_model.transform(rescaledData)
transformed.show(truncate=False)


The topics described by their top-weighted terms:
+-----+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
topic|termIndices |termWeights |
+-----+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0 |[4, 1, 3, 32, 2, 6, 52, 347, 10, 9] |[0.0044242563157922445, 0.004038035015607271, 0.004000129832475176, 0.0034521283994861296, 0.0033924807049772205, 0.0033124317631603004, 0.0032921289083229846, 0.0032328190136729014, 0.003232366680881302, 0.0029894622032362902]|
1 |[0, 8, 4, 788, 160, 41, 1, 6, 795, 3] |[0.00452009729662059, 0.003204275155396905, 0.003154186571600285, 0.00304220026524505, 0.002852600482587961, 0.002820801360152076, 0.002670757154884307, 0.002444226254681802, 0.002431607405936947, 0.002254855951098463] |
2 |[215, 407, 3, 373, 492, 468, 129, 5, 2, 632]|[0.005001385183267786, 0.0037892313680666025, 0.00338679378304917, 0.0032786059377958986, 0.0032675568260248635, 0.0031625571838718807, 0.002818067656729712, 0.002752420594252263, 0.002639680600189169, 0.002595086021481344] |
3 |[0, 2, 1, 5, 8, 3, 6, 40, 9, 16] |[0.005848055567134485, 0.005219164553708182, 0.00503205708678212, 0.004660368149392696, 0.0042885447944648325, 0.0039867635553326895, 0.003934489156306284, 0.003785506728522093, 0.003739865196556462, 0.0036401324494876384] |
4 |[11, 2, 441, 107, 0, 154, 6, 743, 943, 229] |[0.0039254826422052995, 0.003412412740460194, 0.0031730891445389707, 0.0031322492621863036, 0.0027721278479962163, 0.002731394670959251, 0.0026230644954232836, 0.0025947760344385913, 0.002436185603293137, 0.002429385651907412] |
5 |[4, 1, 267, 3, 34, 25, 7, 0, 528, 10] |[0.004859462835938997, 0.0035697631309222415, 0.0035140756612280465, 0.003413662214020629, 0.0030395165879633255, 0.0030204867044609767, 0.003002153880072166, 0.0027610754729658704, 0.0027404553155440776, 0.002618262742232514] |
6 |[0, 4, 275, 1, 5, 735, 9, 6, 489, 85] |[0.005381760192254404, 0.003439146839256086, 0.003309808456181351, 0.0032477627807700874, 0.0029639807403185154, 0.0029460050245914294, 0.0027311128660934404, 0.002589140351214896, 0.0025792806133713275, 0.002544885033336179] |
7 |[34, 0, 106, 5, 117, 13, 7, 22, 2, 4] |[0.00822785174966574, 0.003414068261861523, 0.003192513233606077, 0.0029847538902655784, 0.002668103274665393, 0.002596083273204724, 0.0025480695611883914, 0.002408715643133198, 0.0023568352667459954, 0.002295560376660484] |
8 |[0, 8, 12, 2, 1, 354, 117, 6, 5, 469] |[0.0052155544221318085, 0.004735575798302604, 0.004106626860880221, 0.0036028284937202316, 0.003401365233556418, 0.0031083180751087357, 0.002722260948025578, 0.002631841295097005, 0.002572727572097939, 0.002510860908806942] |
9 |[233, 0, 48, 1, 2, 299, 11, 93, 6, 78] |[0.004364423297468234, 0.0040200334228956995, 0.0035529016742452996, 0.0034472914974302215, 0.0034419716427398733, 0.0033278147453649587, 0.0032708177250902874, 0.0031360219794455764, 0.003134064332080186, 0.003109184624893743]|
+-----+--------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [45]:
pddf = topics.select('termIndices').toPandas()
group1 = []
t=[]
for num in pddf.loc[1][0]:
  #group1.append(vocab[num])
  t.append(num)
#print("Topic 1 most important words are : {}".format(str(group1)))
print(t)

[0, 8, 4, 788, 160, 41, 1, 6, 795, 3]

In [46]:
pddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
termIndices 10 non-null object
dtypes: object(1)
memory usage: 160.0+ bytes

In [47]:
for i in range(10):
  group = []
  for num in pddf.loc[i][0]:
    group.append(vocab[num])
  print("Topic", i+1, " most important words are : {}".format(str(group)))

Topic 1 most important words are : ['cat', 'like', 'get', 'years', 'one', 'love', 'pet', 'turtle', 'got', 'know']
Topic 2 most important words are : ['dog', 'dogs', 'cat', 'boo', 'loves', 'always', 'like', 'love', 'monkey', 'get']
Topic 3 most important words are : ['snakes', 'guinea', 'get', 'ball', 'pig', 'teeth', 'snake', 'im', 'one', 'python']
Topic 4 most important words are : ['dog', 'one', 'like', 'im', 'dogs', 'get', 'love', '3', 'know', 'want']
Topic 5 most important words are : ['cats', 'one', 'ears', 'kitten', 'dog', 'kittens', 'love', 'taylor', 'bath', 'must']
Topic 6 most important words are : ['cat', 'like', 'car', 'get', 'horse', 'back', 'dont', 'dog', 'fucking', 'got']
Topic 7 most important words are : ['dog', 'cat', 'bed', 'like', 'im', 'rip', 'know', 'love', 'meat', 'died']
Topic 8 most important words are : ['horse', 'dog', 'horses', 'im', 'bit', 'time', 'dont', 'also', 'one', 'cat']
Topic 9 most important words are : ['dog', 'dogs', 'people', 'one', 'like', 'killed', 'bit', 'love', 'im', 'gohan']
Topic 10 most important words are : ['bunny', 'dog', 'food', 'like', 'one', 'training', 'cats', 'puppy', 'love', 'videos']

### 4. Identify Video Creators With Cat And Dog Owners In The Audience

In [49]:
#Get all creators whenever the users label is True(cat/dog owner)
df_create = df_pets.select('creator_name').union(pred_all.filter(F.col('prediction') == 1.0).select('creator_name'))

df_create.createOrReplaceTempView("create_table")

#get count
create_count = spark.sql("select distinct creator_name, count(*) as Number\
                          from create_table \
                          group by creator_name \
                          order by Number DESC")

In [50]:
create_count.show()

+--------------------+------+
 creator_name|Number|
+--------------------+------+
 The Dodo| 3141|
 Taylor Nicole Dean| 2563|
 Brave Wilderness| 1988|
 Brian Barczyk| 1887|
Hope For Paws - O...| 1372|
 Robin Seplut| 1292|
 Cole & Marmalade| 1255|
 Gohan The Husky| 1210|
 Vet Ranch| 1046|
 Think Like A Horse| 1011|
 Viktor Larkhill| 966|
Gone to the Snow ...| 949|
 stacyvlogs| 948|
Zak Georges Dog T...| 934|
 RaleighLink14| 669|
 HammyLux| 623|
 Talking Kitty Cat| 532|
 Happy Tails| 492|
 ViralHog| 443|
 Steff J| 432|
+--------------------+------+
only showing top 20 rows

### 5. Results and Future work

#####The topics that cat/dog owners are interested in are summaried as follows:

##### Business opportunities:

##### Future Work:
1. Tune parameters and do cross-validation to improve classifier performance.
2. Explore the users' background info and contents of the videos posted to help increase labelling accuracy.
3. Divide the cat and dog owners to cat owners and dog owners and do some exploration.
4. Try to identify more user segments.